In [ ]:
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")


# 刪除舊檔案
!rm movie-corpus.zip

# 刪除舊資料夾
!rm -rf movie-corpus

# download dataset https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
!wget https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip

# 解壓縮
!unzip movie-corpus.zip

#讀取檔案
file = open("movie-corpus/corpus.json")
data = json.load(file)

corpus = os.path.join("movie-corpus", "")
print(corpus)

rm: cannot remove 'movie-corpus.zip': No such file or directory
--2023-11-29 02:33:48--  https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
Resolving zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)... 128.253.51.179
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40854701 (39M) [application/zip]
Saving to: ‘movie-corpus.zip’

movie-corpus.zip    100%[===================>]  38.96M  49.1MB/s    in 0.8s    

2023-11-29 02:33:49 (49.1 MB/s) - ‘movie-corpus.zip’ saved [40854701/40854701]

Archive:  movie-corpus.zip
   creating: movie-corpus/
  inflating: movie-corpus/utterances.jsonl  
  inflating: movie-corpus/conversations.json  
  inflating: movie-corpus/corpus.json  
  inflating: movie-corpus/speakers.json  
  inflating: movie-corpus/index.json  
movie-corpus/


In [ ]:
def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

file = os.path.join(corpus, "utterances.jsonl")
print(file)
printLines(file, 2)

food = ["banana", "tomato", "apple", "pineapple", "kiwi", "mango", "melon", "lemon"]
print(food[0:7:1]) #第一個數字是起始點, 第二個結束點，第三個是step，每次隔n個元素之後取值

movie-corpus/utterances.jsonl
b'{"id": "L1045", "conversation_id": "L1044", "text": "They do not!", "speaker": "u0", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "not", "tag": "RB", "dep": "neg", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": "L1044", "timestamp": null, "vectors": []}\n'
b'{"id": "L1044", "conversation_id": "L1044", "text": "They do to!", "speaker": "u2", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "to", "tag": "TO", "dep": "dobj", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": null, "timestamp": null, "vectors": []}\n'
['banana']


開始Google SpreadSheet

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/XXXXXXXX/').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


開聲音

In [ ]:
!pip install audio2numpy
import audio2numpy as a2n
audio = open("/content/drive/MyDrive/course_data/sample-3s.mp3","rb")
data = audio.read()
x,sr=a2n.audio_from_file("/content/drive/MyDrive/course_data/sample-3s.mp3")
print(x)
print(sr)

[[-0.04251099 -0.04251099]
 [-0.04415894 -0.04415894]
 [-0.00393677 -0.00393677]
 ...
 [ 0.0133667   0.0133667 ]
 [ 0.01800537  0.01800537]
 [ 0.00500488  0.00500488]]
44100


抓取網站資料

In [ ]:
import requests
news = requests.get("https://udn.com/news/story/6656/7605703")
print(news.text)

# Dcard
#  .then(data=>data.json()).then(data=>{console.log(data)});
#fetch("https://www.dcard.tw/f/fju/p/253570837").then(data=>data.text()).then(data=>console.log(data))

"""
fetch("https://www.dcard.tw/service/api/v2/search/posts?limit=30&query=%E5%B0%8F%E5%A4%A9%E4%BD%BF&highlight=true&sort=relevance&country=TW", {
  "headers": {
    "accept": "*/*",
    "accept-language": "zh-TW,zh;q=0.9",
    "request-through-cf": "true",
    "sec-ch-ua": "\"Chromium\";v=\"118\", \"Google Chrome\";v=\"118\", \"Not=A?Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
  },
  "referrer": "https://www.dcard.tw/search?query=%E6%84%9B%E6%83%85",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "include"
}).then(data=>data.json()).then(data=>{data.map(x=>{console.log(x.id)})});
"""

"""fetch("https://www.dcard.tw/service/api/v2/search/posts?limit=30&query=%E5%B0%8F%E5%A4%A9%E4%BD%BF&highlight=true&sort=relevance&country=TW", {
  "headers": {
    "accept": "*/*",
    "accept-language": "zh-TW,zh;q=0.9",
    "request-through-cf": "true",
    "sec-ch-ua": "\"Chromium\";v=\"118\", \"Google Chrome\";v=\"118\", \"Not=A?Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
  },
  "referrer": "https://www.dcard.tw/search?query=%E6%84%9B%E6%83%85",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "include"
}).then(data=>data.json()).then(data=>{
  data.map(x=>{fetch("https://www.dcard.tw/f/fju/p/"+x.id).then(data=>data.text()).then(data=>console.log(data))})});
"""

<!--getArticle 0.0092 --><!--hottest_news long cache--><!--getReporter 0.0084 --><!--extend_news 0.0033 --><!-- enter if --><!--related_news 0.0574 --><!--content 0.3722 --><!--ad_1 0.0503 --><!--ad 0.2498 --><!--view_type 0.0000 --><!--meta 0.0058 --><!--family_bar 0.0067 --><!--family_bar sub  --><!--breaking_news 0.0077 --><!--top 0.0183 --><!--bottom 0.0056 --><!--nav 0.0000 --><!--code 0.6523 --><!DOCTYPE html>
<html lang="zh-TW" itemscope itemtype="https://schema.org/WebSite">
<head>
<!--
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="X-UA-Compatible" content="ie=edge">
  <meta name="description" content="網頁敘述">
  <meta name="theme-color" content="#f46c00" />
-->
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta http-equiv="content-language" content="zh-Hant-TW">
<meta name="viewport" content="width=device-width, initial

'fetch("https://www.dcard.tw/service/api/v2/search/posts?limit=30&query=%E5%B0%8F%E5%A4%A9%E4%BD%BF&highlight=true&sort=relevance&country=TW", {\n  "headers": {\n    "accept": "*/*",\n    "accept-language": "zh-TW,zh;q=0.9",\n    "request-through-cf": "true",\n    "sec-ch-ua": ""Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"",\n    "sec-ch-ua-mobile": "?0",\n    "sec-ch-ua-platform": ""Windows"",\n    "sec-fetch-dest": "empty",\n    "sec-fetch-mode": "cors",\n    "sec-fetch-site": "same-origin"\n  },\n  "referrer": "https://www.dcard.tw/search?query=%E6%84%9B%E6%83%85",\n  "referrerPolicy": "strict-origin-when-cross-origin",\n  "body": null,\n  "method": "GET",\n  "mode": "cors",\n  "credentials": "include"\n}).then(data=>data.json()).then(data=>{\n  data.map(x=>{fetch("https://www.dcard.tw/f/fju/p/"+x.id).then(data=>data.text()).then(data=>console.log(data))})});\n'

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(news.text, 'html.parser')
data = soup.find("section", {"class": "article-content__editor"})
print(data.text)



國民黨副總統參選人趙少康自登場後，吸引媒體目光，民進黨攻趙搶盡國民黨總統參選人侯友宜鎂光。趙少康說，憲法很清楚，副總統就是備位元首，侯友宜就是總統。趙少康說，他好相處多啦，走路都走在侯後面，聯訪時盡量讓侯講。如果是跟民眾黨主席柯文哲搭檔，侯友宜日子怎麼過？

關於副總統無聲音。趙少康說，民進黨也在搞這點，說侯康當選後，侯友宜變傀儡。他駁斥，憲法很清楚，副總統就是備位元首，現在選舉要贏最重要，他常思考如果選舉沒贏，民進黨繼續執政，台灣完了，大家在討論電台要不要辭職等，台灣變成戰場，什麼都沒了，他很難自外於整個時代，試想中共來台灣會是什麼樣的狀況，所以不能讓中共到台灣，民進黨繼續執政就有可能發生這樣的情況。












趙少康說，侯友宜敢找他，沒想過嗎？表示侯的膽識、肚量滿大，很多時候一人決策很難決策，侯當刑警一路以來也是團隊工作，前總統馬英九在任也常找人去討論。他當下屬時，從來沒有老闆不喜歡他，侯友宜就是總統。

趙少康認為，副總統本來就是備位，人家願意找他商量很好，不願意就是備位，他不認為侯友宜會不接受好建議，如果是他做決策也希望多諮詢。現在他會到處跑是因為在選舉，要勝選，副總統要當作選總統一樣選，但當選後憲法就是憲法，法律就是法律。侯友宜自己也講他不會是備位元首，外交、國防甚至經濟可以一起參與，私下也講這樣，他認為先贏再說。

對於被認為都是趙少康的聲音。趙少康指出，他剛加入，是新人，當然比較有話題性，侯友宜真的很辛苦，從年初開始忙到現在。試想如果不是他，如果今天是侯柯配，侯友宜接下來會怎樣，民眾黨總統參選人柯文哲會怎樣，他好多啦，好相處多啦，走路都走在侯後面，聯訪時盡量讓侯講。如果是柯文哲，侯友宜日子怎麼過？

趙少康強調，很多時候要注意，很多場合要對侯友宜尊重、尊敬。他也提到，中廣董事長沒什麼了不起，辭職就是一封信，有什麼了不起，就是要看國家通訊傳播委員會（NCC）怎麼樣，還有媒體拿他跟民進黨副總統參選人蕭美琴比，是在講什麼，蕭是外交官，拿政府的薪水，她不做還要有人去做，他是民間媒體，他不在還是照樣運作，他當選就是要辭，若希望他辭，民進黨總統參選人賴清德可以恭請侯康配當選。👉2024選戰最前線
 ▪ 政黨票多關鍵？不僅藍綠角力不分區立委 4門檻成小黨生存挑戰

 ▪ 整理包／2024總統與立委選舉保證金知多少？退還門檻一次看

 ▪ 整理包／2

In [ ]:
import jieba

content = data.text
temp = list(jieba.cut(content))
stop_words = ["\n", " ","，","。","（","）","」""","\r\n"]
dictionary = {}
all_words = []
for word in temp:
  if word in stop_words:
    continue

  if word not in all_words:
    all_words.append(word)

  if word in dictionary:
    dictionary[word] = dictionary[word]+1
  else:
    dictionary[word] = 1

print(dictionary)

{'國民黨': 1, '副': 7, '總統': 14, '參選人': 4, '趙': 10, '少康': 9, '自登場': 1, '後': 2, '吸引': 1, '媒體': 3, '目光': 1, '民進': 5, '黨攻': 1, '搶': 1, '盡國民黨': 1, '侯友': 3, '宜鎂光': 1, '說': 6, '憲法': 3, '很': 6, '清楚': 2, '就是': 11, '備位': 5, '元首': 3, '侯友宜': 9, '他': 18, '好': 3, '相處': 2, '多': 3, '啦': 3, '走路': 2, '都': 4, '走': 2, '在': 7, '侯後面': 2, '聯訪': 2, '時': 2, '盡量': 2, '讓': 3, '侯講': 2, '如果': 6, '是': 12, '跟民眾': 1, '黨': 8, '主席': 1, '柯文': 3, '哲搭檔': 1, '日子': 2, '怎麼過': 2, '？': 5, '關於': 1, '無聲音': 1, '也': 6, '搞': 1, '這點': 1, '侯康': 1, '當選': 3, '宜變': 1, '傀儡': 1, '駁斥': 1, '現在': 3, '選舉': 4, '要贏': 1, '最': 2, '重要': 1, '他常': 1, '思考': 1, '沒贏': 1, '繼續': 2, '執政': 2, '台灣': 3, '完': 1, '了': 2, '大家': 1, '討論': 2, '電台': 1, '要': 4, '不要': 1, '辭職': 2, '等': 1, '變成': 1, '戰場': 1, '什麼': 1, '沒': 2, '難': 2, '自外': 1, '於': 2, '整個': 1, '時代': 1, '試想': 2, '中共': 2, '來': 1, '台灣會': 1, '什麼樣': 1, '的': 5, '狀況': 1, '所以': 1, '不能': 1, '到': 3, '就': 1, '有': 3, '可能': 1, '發生': 1, '這樣': 2, '情況': 1, '敢找': 1, '沒想過': 1, '嗎': 1, '表示': 1, '侯': 1, '膽識': 1, '、': 3, '肚量': 1

In [ ]:
import requests
news = requests.get("https://global.udn.com/global_vision/story/8664/3666255?from=udn-referralnews_ch2artbottom")

from bs4 import BeautifulSoup
soup = BeautifulSoup(news.text, 'html.parser')
data = soup.find("div", {"class": "story_body_content"})
print(data.text)
















WhatsApp



twitter




手機醫生與網路病人：英國「遠距醫療」的省錢挑戰

		2019/03/04 吳易澄


用手機「Skype」看醫生，會是英國未來的看病趨勢嗎？圖為英國「掌中家庭醫師」（The GP at Hand）App。 圖／Babylon GP at hand Facebookfacebook

今年年初，英國梅伊（Theresa May）政府公佈了一項NHS（國民醫療保健系統）的十年計畫，計劃重點包括了加強小孩、老人的社區照顧工作，其中一個引起爭議的措施，則是將「數位健康服務」（digital health service）納入NHS未來運作的主流方向之一——未來民眾得以受益於無遠無界的網路系統，打開智慧型手機就能看醫生。

在網路科技發達的今日，透過網路進行遠距看診，似乎是個跟得上時代的創舉，然而這也面對不少批評，包括隔空看診的準確性，以及是否能對弱勢病人帶來實質利益的質疑。













台灣關於數位遠距醫療的討論，除了回應資源的分配問題以外，也聚焦在如何開啟潛在的商機。但是這個在台灣以「開源」的邏輯視之的技術，在英國形成的背景卻是「節流」。

在梅伊政府公布這項政策之際，NHS的執行長西門．史蒂文斯（Simon Stevens）即聲稱：已經沿用70年的醫院約診模式早已過時，網路看診才能夠縮小看病的經濟負擔，並且確保醫療資源的可近性、改善家醫科（GP）診療的品質，同時讓癌症、精神健康與心臟等慢性疾病得到更好的診治。然而，這項美其名為「跟上網路科技時代」的政策，真正的理由或許還是來自預算的短缺。

NHS十年計畫中的「數位健康服務」，引發不少爭議。這項美其名為「跟上網路科技時代」的政策，推展真正的理由，或許還是來自預算的短缺。圖為英國米爾頓凱恩斯大學醫院。 圖／路透社facebook

▌遠距醫療：預算緊縮的解方？

英國的公醫制度舉世聞名，這個「看病免錢」的政策，源於政治理念偏左的工黨在40年代的鼓吹。20世紀中期，英國雖然正值二次戰後百廢待舉、資源短缺之際，NHS卻仍在工黨政府的推動下正式成立。從2012年倫敦奧運開幕典禮上，600多名舞者一同演出「醫護人員大戰病魔」的舞碼，足見英國對這個政策有多麽引以為傲。

不過，在NHS甫上路之際，英國就因戰後嚴重的資金短缺，因此在收費方案上

In [ ]:
dictionary = {}
all_words = []
for word in temp:
  if word in stop_words:
    continue

  if word not in all_words:
    all_words.append(word)

  if word in dictionary:
    dictionary[word] = dictionary[word]+1
  else:
    dictionary[word] = 1

print(dictionary)
print(all_words)

{'國民黨': 1, '副': 7, '總統': 14, '參選人': 4, '趙': 10, '少康': 9, '自登場': 1, '後': 2, '吸引': 1, '媒體': 3, '目光': 1, '民進': 5, '黨攻': 1, '搶': 1, '盡國民黨': 1, '侯友': 3, '宜鎂光': 1, '說': 6, '憲法': 3, '很': 6, '清楚': 2, '就是': 11, '備位': 5, '元首': 3, '侯友宜': 9, '他': 18, '好': 3, '相處': 2, '多': 3, '啦': 3, '走路': 2, '都': 4, '走': 2, '在': 7, '侯後面': 2, '聯訪': 2, '時': 2, '盡量': 2, '讓': 3, '侯講': 2, '如果': 6, '是': 12, '跟民眾': 1, '黨': 8, '主席': 1, '柯文': 3, '哲搭檔': 1, '日子': 2, '怎麼過': 2, '？': 5, '關於': 1, '無聲音': 1, '也': 6, '搞': 1, '這點': 1, '侯康': 1, '當選': 3, '宜變': 1, '傀儡': 1, '駁斥': 1, '現在': 3, '選舉': 4, '要贏': 1, '最': 2, '重要': 1, '他常': 1, '思考': 1, '沒贏': 1, '繼續': 2, '執政': 2, '台灣': 3, '完': 1, '了': 2, '大家': 1, '討論': 2, '電台': 1, '要': 4, '不要': 1, '辭職': 2, '等': 1, '變成': 1, '戰場': 1, '什麼': 1, '沒': 2, '難': 2, '自外': 1, '於': 2, '整個': 1, '時代': 1, '試想': 2, '中共': 2, '來': 1, '台灣會': 1, '什麼樣': 1, '的': 5, '狀況': 1, '所以': 1, '不能': 1, '到': 3, '就': 1, '有': 3, '可能': 1, '發生': 1, '這樣': 2, '情況': 1, '敢找': 1, '沒想過': 1, '嗎': 1, '表示': 1, '侯': 1, '膽識': 1, '、': 3, '肚量': 1

In [ ]:
import jieba

stop_words = ["\n", " ","，","。","（","）","」""","\r\n"]
all_words = []
all = [] #存放所有字典

content = data.text
temp = list(jieba.cut(content))

dictionary = {}
for word in temp:
  if word in stop_words:
    continue

  if word not in all_words:
    all_words.append(word)

  if word in dictionary:
    dictionary[word] = dictionary[word]+1
  else:
    dictionary[word] = 1

all.append(dictionary)
print(all)

[{'WhatsApp': 1, 'twitter': 1, '手機': 7, '醫生': 9, '與': 26, '網路': 19, '病人': 15, '：': 5, '英國': 29, '「': 29, '遠距': 15, '醫療': 29, '的': 181, '省': 1, '錢': 1, '挑戰': 1, '\t': 2, '2019': 1, '/': 2, '03': 1, '04': 1, '吳易': 1, '澄': 1, '用': 1, 'Skype': 1, '看醫生': 1, '會': 1, '是': 14, '未來': 2, '看病': 3, '趨勢': 3, '嗎': 1, '？': 4, '圖為': 4, '掌中': 2, '家庭': 5, '醫師': 5, 'The': 2, 'GP': 6, 'at': 3, 'Hand': 2, 'App': 1, '圖': 9, '／': 9, 'Babylon': 1, 'hand': 1, 'Facebookfacebook': 1, '今年年初': 1, '英國梅': 1, '伊': 1, 'Theresa': 1, 'May': 1, '政府': 9, '公': 1, '佈': 1, '了': 19, '一項': 2, 'NHS': 28, '國民醫療': 1, '保健': 1, '系統': 6, '十年': 2, '計畫': 2, '計劃': 1, '重點': 2, '包括': 4, '加強': 1, '小孩': 1, '、': 18, '老人': 3, '社區': 2, '照顧': 1, '工作': 2, '其中': 1, '一個': 7, '引起': 1, '爭議': 2, '措施': 1, '則是將': 1, '數位': 9, '健康': 5, '服務': 2, 'digital': 1, 'health': 1, 'service': 1, '納入': 1, '運作': 1, '主流': 1, '方向': 1, '之一': 1, '—': 2, '未來民眾': 1, '得以': 1, '受益': 1, '於': 14, '無遠': 1, '無界': 1, '打開': 1, '智慧型': 2, '就': 4, '能': 5, '看': 5, '在': 37, '科技': 16, 

In [ ]:
#https://github.com/fxsjy/jieba
import jieba
import jieba.analyse
text = open("/content/drive/MyDrive/course_data/data.txt").read()

#不要的字
stop_words = [
    "我","他","i","you","，","於", "\n"
]

#判別是否是不要的字
def remove_stop_words(word):
  return (word not in stop_words)

my_dict = [
    "8月 5",
    "殺警案 5"
    ]
with open("dict.txt", "w") as f:
  for word in my_dict:
    f.write(word+"\n")
jieba.load_userdict("dict.txt")

result = jieba.cut(text)
result = list(result)

#過濾掉不要的字
after_filter = filter(remove_stop_words, result)

#將過濾的結果轉為列表
result = list(after_filter)

#印出結果
print(result)
print(result.count("外交"))

with open("dict.txt", "a") as f:
  for word in result:
    f.write(word+" 2\n")


!rm wordf.csv
all_words = set(result)
with open("wordf.csv", "a") as f:
  for word in all_words:
    if len(word) > 1:
      f.write(str(result.count(word))+";"+word+";\n")



['外交', '兩岸', ' ', '賴清德', '賴在', '7', '/', '5', '以', '「', '維護', '台海', '和平', '方案', '」', '為題', '投書', '《', '華爾街', '日報', '》', '表態', '支持', '維持', '兩岸', '現狀', '主張', '維持', '現狀', '是', '符合', '「', '中華民國', '台灣', '」', '與', '國際', '社會', '的', '最佳', '利益', '。', ' ', '提出', '保衛', '和平', '的', '四大', '支柱', '包括', '強化國', '防威懾力', '、', '提升', '經濟', '安全', '、', '與', '全球', '民主', '國家', '建立', '伙伴', '關', '係', '、', '要', '有', '穩定', '且', '具有', '原則', '的', '兩岸', '領導力', '。', ' ', '侯友宜', ' ', '侯在', '6', '/', '23', '桃園', '與', '國民黨', '主席', '朱立倫', '一起', '參香時', '提出', '「', '兩岸', '一家人', '」', '的', '論述', '並', '表示', '兩岸', '要', '在', '「', '對', '等', '」', '、', '「', '尊嚴', '」', '、', '「', '友善', '」', '底下', '和平', '交流', '重', '啟務', '實來', '往', '讓', '兩岸', '更加', '穩定', '。', ' ', '此外', '侯友宜', '在', '7', '/', '3', '接受', '廣播', '節目', '專訪', '時', '表態', '支持', '合乎', '「', '中華民國', '憲法', '」', '的', '「', '九二', '共識', '」', ' ', '主張', '兩岸', '「', '互不', '承', '認主權', '互不', '否', '認治權', '」', '的', '政治', '現實', '。', ' ', '柯文', '哲', ' ', '柯在', '7', '/', '18', '接受', '媒體訪', '問時', 

第一篇文章

In [ ]:
import jieba

#定義好停用詞、所有字的字典、所有文件的字典
stop_words = ["\n", " ","，","。","（","）","」""","\r\n"]
all_words = []
all = [] #存放所有字典

#抓第一篇文章
import requests
news = requests.get("https://udn.com/news/story/7266/7605572?from=redpush")


from bs4 import BeautifulSoup
soup = BeautifulSoup(news.text, 'html.parser')
data = soup.find("section", {"class": "article-content__editor"})


content = data.text
temp = list(jieba.cut(content))

dictionary = {}
for word in temp:
  if word in stop_words:
    continue

  if word not in all_words:
    all_words.append(word)

  if word in dictionary:
    dictionary[word] = dictionary[word]+1
  else:
    dictionary[word] = 1

all.append(dictionary)
print(all)

[{'我國': 2, '於': 7, '1988': 1, '年': 6, '全面': 1, '執行': 1, '醫院': 22, '評鑑': 9, '迄今': 1, '30': 1, '多年': 1, '一向': 1, '只升': 1, '不降': 1, '但': 2, '今年': 2, '卻': 1, '可能': 2, '出現': 1, '史上': 2, '首次': 2, '現有': 1, '醫學': 6, '中心': 15, '中箭': 1, '落馬': 1, '醫界': 2, '近日': 2, '盛傳': 1, '台北': 14, '慈濟': 6, '準醫學': 7, '擠下': 1, '萬芳': 4, '晉升': 3, '對此': 1, '慈濟院': 2, '長': 2, '趙': 5, '有': 7, '誠': 5, '受訪': 1, '時間': 2, '接證實': 1, '這項': 1, '傳聞': 1, '他': 2, '說': 3, '「': 4, '日前': 1, '聽到': 1, '訊息': 1, '時': 1, '心裡': 1, '非常': 1, '激動': 1, '直到': 1, '這兩天': 1, '才能': 1, '比較': 1, '平': 1, '復': 1, '地': 1, '話': 1, '全國': 1, '共有': 1, '19': 1, '家醫學': 1, '依規定': 1, '每': 1, '4': 1, '一次': 1, '因新冠': 1, '疫情': 3, '隔': 1, '了': 4, '7': 1, '為': 4, '此': 2, '被': 2, '喻為': 1, '最': 1, '煎熬': 1, '區': 1, '宜蘭縣': 1, '、': 16, '基隆市': 1, '新北市': 1, '台北市': 1, '連江縣': 1, '金門縣': 1, '11': 1, '月': 1, '18': 1, '日起': 1, '完成': 2, '第二': 1, '階段': 1, '列為': 1, '末段': 1, '班的則': 1, '幾家': 1, '提心': 1, '吊膽': 1, '而雙': 2, '和': 2, '等': 3, '再次': 1, '挑戰': 3, '莫不': 1, '關注': 1, '誰': 2, '

第二篇文章

In [ ]:
import requests
news = requests.get("https://global.udn.com/global_vision/story/8664/3666255?from=udn-referralnews_ch2artbottom")

from bs4 import BeautifulSoup
soup = BeautifulSoup(news.text, 'html.parser')
data = soup.find("div", {"class": "story_body_content"})

content = data.text
temp = list(jieba.cut(content))
dictionary = {}
for word in temp:
  if word in stop_words:
    continue

  if word not in all_words:
    all_words.append(word)

  if word in dictionary:
    dictionary[word] = dictionary[word]+1
  else:
    dictionary[word] = 1

all.append(dictionary)
print(all)

[{'我國': 2, '於': 7, '1988': 1, '年': 6, '全面': 1, '執行': 1, '醫院': 22, '評鑑': 9, '迄今': 1, '30': 1, '多年': 1, '一向': 1, '只升': 1, '不降': 1, '但': 2, '今年': 2, '卻': 1, '可能': 2, '出現': 1, '史上': 2, '首次': 2, '現有': 1, '醫學': 6, '中心': 15, '中箭': 1, '落馬': 1, '醫界': 2, '近日': 2, '盛傳': 1, '台北': 14, '慈濟': 6, '準醫學': 7, '擠下': 1, '萬芳': 4, '晉升': 3, '對此': 1, '慈濟院': 2, '長': 2, '趙': 5, '有': 7, '誠': 5, '受訪': 1, '時間': 2, '接證實': 1, '這項': 1, '傳聞': 1, '他': 2, '說': 3, '「': 4, '日前': 1, '聽到': 1, '訊息': 1, '時': 1, '心裡': 1, '非常': 1, '激動': 1, '直到': 1, '這兩天': 1, '才能': 1, '比較': 1, '平': 1, '復': 1, '地': 1, '話': 1, '全國': 1, '共有': 1, '19': 1, '家醫學': 1, '依規定': 1, '每': 1, '4': 1, '一次': 1, '因新冠': 1, '疫情': 3, '隔': 1, '了': 4, '7': 1, '為': 4, '此': 2, '被': 2, '喻為': 1, '最': 1, '煎熬': 1, '區': 1, '宜蘭縣': 1, '、': 16, '基隆市': 1, '新北市': 1, '台北市': 1, '連江縣': 1, '金門縣': 1, '11': 1, '月': 1, '18': 1, '日起': 1, '完成': 2, '第二': 1, '階段': 1, '列為': 1, '末段': 1, '班的則': 1, '幾家': 1, '提心': 1, '吊膽': 1, '而雙': 2, '和': 2, '等': 3, '再次': 1, '挑戰': 3, '莫不': 1, '關注': 1, '誰': 2, '

製作資料集

In [ ]:
dataset = [] #建立資料集列表

for dictionary in all: #依序取出所有字典
  temp = [] #建立一個暫存列表
  for word in all_words: #依序取出所有字
    if word in dictionary: #如果這個字在這份文件的列表中
      temp.append(dictionary[word]) #就添加上其出現的次數
    else: #如果沒有這個字
      temp.append(0) #就填入0
  dataset.append(temp) #最後添加到資料集列表

print(dataset)

import pandas as pd  #引入pandas
df = pd.DataFrame(dataset, columns=all_words) #建立資料框架
df.to_csv("dataset.cav") #存成csv

[[2, 7, 1, 6, 1, 1, 22, 9, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 6, 15, 1, 1, 2, 2, 1, 14, 6, 7, 1, 4, 3, 1, 2, 2, 5, 7, 5, 1, 2, 1, 1, 1, 2, 3, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4, 1, 4, 2, 2, 1, 1, 1, 1, 1, 16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 3, 1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 2, 3, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 1, 2, 2, 2, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 6, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

feature selection

In [ ]:
selection = ["醫院", "醫學", "醫界"]
df[selection]
df["hostpital"] = df['醫院'] > 10
df

,我國,於,1988,年,全面,執行,醫院,評鑑,迄今,30,...,穩定,條件,我們,收看,文章,請訂,閱轉,角國際,專頁,hostpital
0,2,7,1,6,1,1,22,9,1,1,...,0,0,0,0,0,0,0,0,0,True
1,0,14,0,13,0,0,6,0,0,0,...,1,1,1,1,1,1,1,1,1,False


In [ ]:
# Splits each line of the file to create lines and conversations
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Extract fields for line object
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # Extract fields for conversation object
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [ ]:
with open("data/utterances.jsonl", 'r', encoding='iso-8859-1') as f:
  for line in f:
    lineJson = json.loads(line)
    lineObj = {}
    lineObj["lineID"] = lineJson["id"]
    lineObj["characterID"] = lineJson["speaker"]
    lineObj["text"] = lineJson["text"]
    print(lineObj)


串流輸出內容已截斷至最後 5000 行。
{'lineID': 'L649417', 'characterID': 'u8880', 'text': 'You could fly down tomorrow.  I could be there the next day  I think I can last that long without you.'}
{'lineID': 'L649416', 'characterID': 'u8879', 'text': ''}
{'lineID': 'L649415', 'characterID': 'u8880', 'text': 'Michael, do you want to wear those for the rest of your life...?'}
{'lineID': 'L649414', 'characterID': 'u8879', 'text': "God knows, I'm intriguing."}
{'lineID': 'L649413', 'characterID': 'u8880', 'text': "I called Dr. Styles at the Neurological Institute  In L.A., remember...?  He said he'd see you.  He was intrigned."}
{'lineID': 'L649412', 'characterID': 'u8880', 'text': "Michael - this place is increadible.  You - you're brilliant...  The things you created..."}
{'lineID': 'L649411', 'characterID': 'u8879', 'text': 'Seems more like a year than a month.'}
{'lineID': 'L649407', 'characterID': 'u8879', 'text': "You stop it.  you're loving, and gentel..  You're the most remarkable woman I've ever 